# t-SNE (Student t's distributed Stochastic Neighbor Embedding)

## Model Specification

The main purpose of t-SNE is visualization of high-dimensional data. Hence, it works best when the data will be embedded on two or three dimensions

Like SNE, t-SNE also starts by converting the high-dimensional Euclidean distances between datapoints into conditional probabilities that represents probabilities. Mathematically, the conditional probability $p_{ij}$ is given by $p_{ij}=\frac{p_{i|j}+p_{j|i}}{2n}$, where
$$p_{i|j}=\frac{\exp(-|x_i-x_j|^2/\sigma_i^2)}{\sum_{k\neq i}\exp(-|x_k-x_i|^2/\sigma_i^2)},$$ where $\sigma_i$ is the standard deviation of the Gaussian around data set $i$. And we set $p_{ii}=0$. The purpose of setting the symmetric $p_{ij}$ in this way is so that $\sum_jp_{ij}>\frac{1}{n}$ for all datapoints $x_i$, and $x_i$ makes a significant contribution to the cost function.

We model the similarity of the mapped points by 
$$q_{ij}=\frac{(1+|y_i-y_j|^2)^{-1}}{\sum_{k\neq l}(1+|y_k-y_l|^2)^{-1}}.$$ This is a Student t's distribution (actually a Cauchy distribution). Again we set $q_{ii}=0$. 

The idea is if the mapped points $y$ correctly preserves the similarity between $x$, the conditional probabilities $p$ and $q$ should be close to equal. 

### Variants and Generalizations

## Theoretical Properties

### Advantages

- t-SNE is much easier to optimize than SNE.
- t-SNE produces significantly better visualization results by reducing the tendency to crowd points together in the center of the map. 
- t-SNE is better than existing techniques at revealing structures at different scales, such as points lying in different, multiple manifolds or clusters.

### Disadvantages

- The Barnes-Hut t-SNE method, although runs quicker, is limited to two or three dimensional embeddings. But probably this is not much of an issue, since visualization is done in 2-3 dimensions anyways.

### Relation to Other Models

t-SNE is a generalization to Stochastic Neighboring Embedding, by using (1) the Student t's distribution in modeling the embedded in low dimension and (2) symmetric version of the SNE cost function. This brings about the benefit of avoiding crowding, as well as revealing fine structures at different scales. 

  - The Student t's distribution in modeling the embedded in low dimension helps solve the crowding problem. Intuitively, the area of the two-dimensional map that is available to accomodate moderately distant datapoints will not be nearly enough compared with the area available to accomodate nearby datapoints. That is why representations of high-dimensional data get crushed and overlap each other. With a heavy tail of the Student t's distribution, a moderate space in the high-dimensional space is represented by much larger distance in the low-dimensional map.
  
  - The symmetric version of cost function has simpler gradients, which helps the optimization in training.

## Empirical Performance

### Advantages 

### Disadvantages

- t-SNE is computationally expensive, and can take several hours on million-sample datasets where PCA will finish in seconds or minutes
- The algorithm is stochastic and multiple restarts with different seeds can yield different embeddings. However, it is perfectly legitimate to pick the embedding with the least error. In addition to randomness, different hyperparameters will also give rise to different results. Thus multiple runs may be needed to ascertain whether certain patterns/topology persist. 
- Global structure is not explicitly preserved. This is problem is mitigated by initializing points with PCA (using `init=’pca’`; see implementation below).
- **Cluster sizes are distorted**. One cannot intuitively evaluate cluster size for standard deviation, dispersion or any other similar measures. This is because t-SNE expands denser clusters and contracts sparser ones to even out cluster sizes, and thus cluster sizes are distorted. But t-SNE needs this distortion for the crisp and clear plots it produces.

## Implementation Details and Practical Tricks

**Implementation in sklearn**

It is highly recommended to use another dimensionality reduction method (e.g. PCA for dense data or TruncatedSVD for sparse data) to reduce the number of dimensions to a reasonable amount (e.g. 50) if the number of features is very high. This will suppress some noise and speed up the computation of pairwise distances between samples. For more tips see Laurens van der Maaten’s FAQ

The implementation in sklearn uses the Barnes-Hut method, which is not read in much details yet.

In [4]:
import numpy as np
from sklearn.manifold import TSNE
X = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
# fit_transform: Fit X into an embedded space and return that transformed output
tsne = TSNE(n_components=2, 
            perplexity=30.0, 
            early_exaggeration=12.0, 
            learning_rate=200.0, 
            n_iter=1000, 
            init='random', 
            method='barnes_hut', 
            angle=0.5)
X_embedded = tsne.fit_transform(X)
X_embedded

array([[ -1.02245119e-04,   7.20309540e-05],
       [  6.99963016e-05,   3.47819822e-05],
       [  5.96248693e-06,  -9.36522420e-05],
       [  4.08332454e-07,   3.16607053e-05]])

** Some commonly used inputs**

**`n_components`**: Dimension of the (lower-dimensional) embedded space.

**`perplexity`**: The perplexity is related to the number of nearest neighbors that is used in other manifold learning algorithms. Larger datasets usually require a larger perplexity. Consider selecting a value between 5 and 50. The choice is not extremely critical since t-SNE is quite insensitive to this parameter.

**early_exaggeration**: Controls how tight natural clusters in the original space are in the embedded space and how much space will be between them. For larger values, the space between natural clusters will be larger in the embedded space. Again, the choice of this parameter is not very critical. If the cost function increases during initial optimization, the early exaggeration factor or the learning rate might be too high.

**learning_rate**: The learning rate for t-SNE is usually in the range [10.0, 1000.0]. If the learning rate is too high, the data may look like a ‘ball’ with any point approximately equidistant from its nearest neighbours. If the learning rate is too low, most points may look compressed in a dense cloud with few outliers. If the cost function gets stuck in a bad local minimum increasing the learning rate may help.

**n_iter**: Maximum number of iterations for the optimization. Should be at least 250.

**init**: Initialization of embedding. Possible options are ‘random’, ‘pca’, and a numpy array of shape (n_samples, n_components). PCA initialization cannot be used with precomputed distances and is usually more globally stable than random initialization.

**Optimizing t-SNE**

(This section is taken from [sklearn material](http://scikit-learn.org/stable/modules/manifold.html#t-distributed-stochastic-neighbor-embedding-t-sne). But you need to first take a look at Barnes-Hut method to understand.)

**Barnes-Hut t-SNE**

(This section is taken from [sklearn material](http://scikit-learn.org/stable/modules/manifold.html#t-distributed-stochastic-neighbor-embedding-t-sne). But you need to first take a look at Barnes-Hut method to understand.)

## Use Cases

## Results Interpretation, Metrics and Visualization

## References 

- van der Maaten, L.J.P.; Hinton, G.E. Visualizing High-Dimensional Data
Using t-SNE. Journal of Machine Learning Research 9:2579-2605, 2008.
- [scikit-learn Document 2.2.9](http://scikit-learn.org/stable/modules/manifold.html#t-distributed-stochastic-neighbor-embedding-t-sne)

### Further Reading

- van der Maaten, L.J.P. [t-Distributed Stochastic Neighbor Embedding](https://lvdmaaten.github.io/tsne/): contains FAQ that provide more tips in practice
- L.J.P. van der Maaten. [Accelerating t-SNE using Tree-Based Algorithms](http://lvdmaaten.github.io/publications/papers/JMLR_2014.pdf).
Journal of Machine Learning Research 15(Oct):3221-3245, 2014. : variants on the Barnes-Hut
- [“How to Use t-SNE Effectively”](https://distill.pub/2016/misread-tsne/) provides a good discussion of the effects of the various parameters, as well as interactive plots to explore the effects of different parameters.

## Misc.